In [1]:
'''
from pandas import *
xls = ExcelFile('path_to_file.xls')
df = xls.parse(xls.sheet_names[0])
print df.to_dict()

'''

"\nfrom pandas import *\nxls = ExcelFile('path_to_file.xls')\ndf = xls.parse(xls.sheet_names[0])\nprint df.to_dict()\n\n"

In [2]:
#imports
import pandas as pd
import numpy as np
#from mip import *
import gurobipy as gp

In [3]:
#max attendance at classes= 45
subject_and_teacher, attendance_at_class = gp.multidict({
    'Bible_Roni': 33,
    'Bible_Rachel': 33,
    'Literature_Sharon': 20,
    'Literature_Liora': 20,
    'Math_Yuri': 38,
    'Math_Sarit': 38,
    'Math_Victor': 38,
    'English_Miriam': 22,
    'English_Haim': 22,
    'Physics_Tzvika': 10,
    'Language_Hanita': 12,
    'Language_Smadar': 12,
    'Geography_Ilana': 40,
    'History_Ruth': 31,
    'History_Etti': 31,
    'Chemistry_Shelly': 40,
    'CitizenshipLesson_Tami': 18,
    'Economics_Menachem': 44,
    'Arabic_Hanita': 9
})

In [4]:
# class + avg grade
student_avg = gp.multidict({
    'Bible_Roni': 70,
    'Bible_Rachel': 70,
    'Literature_Sharon': 72,
    'Literature_Liora': 72,
    'Math_Yuri': 53,
    'Math_Sarit': 53,
    'Math_Victor': 53,
    'English_Miriam': 68,
    'English_Haim': 68,
    'Physics_Tzvika': 41,
    'Language_Hanita': 80,
    'Language_Smadar': 80,
    'Geography_Ilana': 94,
    'History_Ruth': 78,
    'History_Etti': 78,
    'Chemistry_Shelly': 75,
    'CitizenshipLesson_Tami': 86,
    'Economics_Menachem': 56,
    'Arabic_Hanita': 66
})[1]

In [5]:
teacher_free_time = {
    'Sunday': [['Bible_Rachel', 10, 12], ['Literature_Liora', 14, 16]],
    'Monday': [['Literature_Sharon', 13, 15], ['Math_Yuri', 12, 14], ['English_Haim', 17, 19]],
    'Tuesday': [['Physics_Tzvika', 16, 18], ['Language_Hanita', 14, 16],
                ['Economics_Menachem', 14, 16], ['English_Miriam', 10, 12]],
    'Wednesday': [['Language_Smadar', 9, 11], ['Geography_Ilana', 13, 15],
                  ['History_Ruth', 12, 14], ['CitizenshipLesson_Tami', 15, 17]],
    'Thursday': [['Math_Sarit', 9, 11], ['Math_Victor', 17, 19], ['Arabic_Hanita', 18, 20]],
    'Friday': [['Bible_Roni', 8, 10], ['History_Etti', 10, 12], ['Chemistry_Shelly', 11, 13]]
}

In [6]:
student_free_time = {
    'Sunday': [14,20],
    'Monday': [12,15],
    'Tuesday': [16,19],
    'Wednesday': [12,17],
    'Thursday': [17,20],
    'Friday': [8,13]
}

In [7]:
def get_collisions(course_times):
    # Understand which courses collide
    collisions = []
    for day_courses in course_times.values():
        # day_courses is a list of all courses in a specific day
        if len(day_courses) == 1:
            continue  # no possible collisions

        # Compare current course with all courses after it
        for i, course in enumerate(day_courses):
            if i == len(day_courses) - 1:
                continue  # all collisions were already specified before
            for other_course in day_courses[i + 1:]:
                start_time_current = course[1]
                end_time_current = course[2]
                start_time_other = other_course[1]
                course_name = course[0]
                other_course_name = other_course[0]
                if (start_time_other < end_time_current) or (start_time_other == start_time_current):
                    collisions.append((course_name, other_course_name))

    return collisions

collisions = get_collisions(teacher_free_time)

In [8]:
print(collisions)

[('Literature_Sharon', 'Math_Yuri'), ('Physics_Tzvika', 'Language_Hanita'), ('Physics_Tzvika', 'Economics_Menachem'), ('Physics_Tzvika', 'English_Miriam'), ('Language_Hanita', 'Economics_Menachem'), ('Language_Hanita', 'English_Miriam'), ('Economics_Menachem', 'English_Miriam'), ('Geography_Ilana', 'History_Ruth'), ('Math_Victor', 'Arabic_Hanita'), ('History_Etti', 'Chemistry_Shelly')]


In [9]:
def get_bad_classes(teacher_free_time, student_free_time):

    bad_classes = []

    for (student_day, s_free_time), (day, day_subjects) in zip(student_free_time.items(), teacher_free_time.items()):
    #for student_day, s_free_time in student_free_time.items(): 
        day_length = student_free_time[student_day]     
        
        #for day, day_subjects in teacher_free_time.items():

        for subject in day_subjects:
            start = subject[1]
            end = subject[2]
            if start < day_length[0] or end > day_length[1]:
                bad_classes.append(subject[0])

    return bad_classes


bad_classes = get_bad_classes(teacher_free_time, student_free_time)

In [10]:
print(bad_classes)

['Bible_Rachel', 'English_Haim', 'Language_Hanita', 'Economics_Menachem', 'English_Miriam', 'Language_Smadar', 'Math_Sarit']


In [11]:
def get_same_classes(teacher_free_time):
    same_subjects = {}
    #subject --> ['Bible_Rachel', 10, 12]
    for day in teacher_free_time.values():
        for subject in day:
            #subject[0] --> Bible_Rachel
            split_sub_and_teacher = subject[0].split("_")
            #split_course_name --> Bible,Rachel
            if split_sub_and_teacher[0] in same_subjects:
                same_subjects[split_sub_and_teacher[0]].append(subject)
            else:
                same_subjects[split_sub_and_teacher[0]] = [subject]
    return same_subjects

same_subjects = get_same_classes(teacher_free_time)

In [12]:
print(same_subjects)

{'Bible': [['Bible_Rachel', 10, 12], ['Bible_Roni', 8, 10]], 'Literature': [['Literature_Liora', 14, 16], ['Literature_Sharon', 13, 15]], 'Math': [['Math_Yuri', 12, 14], ['Math_Sarit', 9, 11], ['Math_Victor', 17, 19]], 'English': [['English_Haim', 17, 19], ['English_Miriam', 10, 12]], 'Physics': [['Physics_Tzvika', 16, 18]], 'Language': [['Language_Hanita', 14, 16], ['Language_Smadar', 9, 11]], 'Economics': [['Economics_Menachem', 14, 16]], 'Geography': [['Geography_Ilana', 13, 15]], 'History': [['History_Ruth', 12, 14], ['History_Etti', 10, 12]], 'CitizenshipLesson': [['CitizenshipLesson_Tami', 15, 17]], 'Arabic': [['Arabic_Hanita', 18, 20]], 'Chemistry': [['Chemistry_Shelly', 11, 13]]}


In [13]:
import time
import datetime
start = time.time()
start = datetime.datetime.now()

# Creating the model
model = gp.Model("Partani")

# Create decision variables for the subjects the student should take
subjects_to_assign = model.addVars(subject_and_teacher, name="Subjects+Teachers", vtype=gp.GRB.BINARY)


Restricted license - for non-production use only - expires 2022-01-13


In [14]:

# objective
# The objective is to minimize (lowest grades and min attendance)
model.ModelSense = gp.GRB.MAXIMIZE
##########################################################################################################       CHECK!
attendance_objective = subjects_to_assign.prod(attendance_at_class)
attendance_priority = 1
attendance_weight = 0.5

student_avg_objective = subjects_to_assign.prod(student_avg)
student_avg_priority = 0
student_avg_weight = 1

model.setObjectiveN(attendance_objective, attendance_priority, weight=attendance_weight)
model.setObjectiveN(student_avg_objective, student_avg_priority, weight=student_avg_weight)


In [15]:
'''
# objective
# The objective is to minimize (lowest grades and min attendance)
model.ModelSense = gp.GRB.MAXIMIZE

w1= 1 #weights for grades
w2= 0.5 #weights for attendance
Cost = 0
Cost += (gp.quicksum(w1*student_avg[s] + w2*attendance_at_class[s] for s in subject_and_teacher))

model.setObjective(Cost)
'''

'\n# objective\n# The objective is to minimize (lowest grades and min attendance)\nmodel.ModelSense = gp.GRB.MAXIMIZE\n\nw1= 1 #weights for grades\nw2= 0.5 #weights for attendance\nCost = 0\nCost += (gp.quicksum(w1*student_avg[s] + w2*attendance_at_class[s] for s in subject_and_teacher))\n\nmodel.setObjective(Cost)\n'

In [16]:
def optimize(subjects_to_assign, same_subjects, bad_classes, collisions):

    for course1, course2 in collisions:
        model.addConstr(subjects_to_assign.sum(course1) + subjects_to_assign.sum(course2) <= 1)

    for course in bad_classes:
        model.addConstr(subjects_to_assign.sum(course) == 0)

    for course in same_subjects.values():
        courses_sum = 0
        for course_time in course:
            courses_sum += subjects_to_assign.sum(course_time)
        model.addConstr(courses_sum <= 1)

    model.optimize()

    for v in model.getVars():
        print('%s %g' % (v.varName, v.x))

    print('Obj: %g' % model.objVal)
    end = time.time()
    end = datetime.datetime.now()
    print(end - start)
    
optimize(subjects_to_assign, same_subjects, bad_classes, collisions)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 29 rows, 19 columns and 46 nonzeros
Model fingerprint: 0x1ab82227
Variable types: 0 continuous, 19 integer (19 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+00, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 29 rows, 19 columns and 46 nonzeros
Variable types: 0 continuous, 19 integer (19 binary)
Coefficient statistic

In [17]:
'''


# Model
model = gp.Model("Partani")

# Create decision variables for the foods to buy
sub_to_assign = m.addVars(subject_and_teacher, name="sub+teach")

# You could use Python looping constructs and m.addVar() to create
# these decision variables instead.  The following would be equivalent
#
# buy = {}
# for f in foods:
#   buy[f] = m.addVar(name=f)

# The objective is to minimize the costs
model.setObjective(sub_to_assign.prod(cost), GRB.MINIMIZE)

# Using looping constructs, the preceding statement would be:
#
# m.setObjective(sum(buy[f]*cost[f] for f in foods), GRB.MINIMIZE)

# Nutrition constraints
m.addConstrs((gp.quicksum(nutritionValues[f, c] * buy[f] for f in foods)
             == [minNutrition[c], maxNutrition[c]]
             for c in categories), "_")
             
             
             
'''

'\n\n\n# Model\nmodel = gp.Model("Partani")\n\n# Create decision variables for the foods to buy\nsub_to_assign = m.addVars(subject_and_teacher, name="sub+teach")\n\n# You could use Python looping constructs and m.addVar() to create\n# these decision variables instead.  The following would be equivalent\n#\n# buy = {}\n# for f in foods:\n#   buy[f] = m.addVar(name=f)\n\n# The objective is to minimize the costs\nmodel.setObjective(sub_to_assign.prod(cost), GRB.MINIMIZE)\n\n# Using looping constructs, the preceding statement would be:\n#\n# m.setObjective(sum(buy[f]*cost[f] for f in foods), GRB.MINIMIZE)\n\n# Nutrition constraints\nm.addConstrs((gp.quicksum(nutritionValues[f, c] * buy[f] for f in foods)\n             == [minNutrition[c], maxNutrition[c]]\n             for c in categories), "_")\n             \n             \n             \n'